In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

<div class="alert alert-warning">

# PS 5 - Bayesian Inference: The Coin of Infinite Hypotheses

In this problem set, we will explore **Bayesian inference** — a principled framework for updating beliefs based on evidence. We'll use a simple example: estimating the bias of a coin from observed flips.

## The Story

One day, you find yourself standing in a musty room in the back of an old magic shop. Heavy velvet curtains cover the dirty windows, reluctantly letting in a few narrow beams of light, which succeed only in illuminating layers of dust suspended in the stale air. You glance around cautiously, trying not to invite scrutiny from the watchful shopkeep. Your eyes dance over row upon row of leather-bound books, resting on the rich mahogany shelves lining each wall.

Before you can even turn around, the shopkeep anticipates your question. "You've come for a coin, haven't you?" His voice sounds strangely distant. You nod, swallowing. He totters past you, one of his legs struggling to keep up with the other. Reaching a black armoire in the corner of the room — how did you not notice it before? — he stops. The shopkeep slowly opens one of the drawers, revealing a beautiful, glimmering coin.

Your father's deep, soulful voice echoes in your head: *"Fetch me a coin of Azeroth, child, but only if its probability of landing heads is between 0.55 and 0.75."*

---

**Is this coin what father asked for?** How can we determine if this coin matches his request?

## The Problem

Let $\theta$ denote the coin's probability of landing heads on each toss. Our task is to **infer** $\theta$ from observed data. This is a learning problem where:

- **Hypothesis space**: All possible values of $\theta$ from 0 to 1 (infinitely many hypotheses!)
- **Data**: A sequence of coin flips (heads = 1, tails = 0)
- **Goal**: Estimate $\theta$ as accurately as possible

We'll compare three estimation methods:
1. **Maximum Likelihood Estimation (MLE)**: Use only the observed data
2. **Maximum A Posteriori (MAP)**: Combine data with prior beliefs
3. **Posterior Mean**: Average over all hypotheses weighted by their posterior probability

</div>

---

# 1. Maximum Likelihood Estimation (MLE)

The simplest approach is **Maximum Likelihood Estimation**: find the value of $\theta$ that makes the observed data most probable.

For coin flips, if we observe $k$ heads in $n$ flips, the likelihood of this data given $\theta$ is:

$$P(\text{data} | \theta) = \theta^k (1-\theta)^{n-k}$$

The MLE estimate is the $\theta$ that maximizes this likelihood. Using calculus, this turns out to be simply:

$$\hat{\theta}_{MLE} = \frac{k}{n} = \frac{\text{number of heads}}{\text{total flips}}$$

**Key property**: MLE uses only the observed data — no prior beliefs factor in.

<div class="alert alert-success">

## Exercise 1A

Implement a function that computes the MLE estimate of $\theta$ from a sequence of coin flips.

</div>

In [ ]:
def mle_estimate(sequence):
    """Compute the Maximum Likelihood Estimate of theta.
    
    Parameters
    ----------
    sequence : numpy array of 1s and 0s
        The observed sequence of coin flips (1 = heads, 0 = tails)
        
    Returns
    -------
    float
        The MLE estimate of theta (probability of heads)
    """
    # YOUR CODE HERE

In [ ]:
# Test your implementation
assert mle_estimate(np.array([1, 1, 0, 1, 0, 1, 1, 1, 1, 1])) == 0.8
assert mle_estimate(np.array([1, 1, 1, 1, 1])) == 1.0
assert mle_estimate(np.array([0, 0, 0, 0, 0])) == 0.0
assert mle_estimate(np.array([1, 0])) == 0.5
assert mle_estimate(np.array([1, 0, 1, 0, 1, 0])) == 0.5
print("All tests passed!")

<div class="alert alert-success">

## Exercise 1B

The code below plots how the MLE estimate evolves as we observe more flips. Two sequences are shown:
- One sequence is all heads (MLE converges to 1.0)
- The other is all tails (MLE converges to 0.0)

**Your task**: Add axis labels, a title, and a legend to make this a complete, interpretable figure.

</div>

In [ ]:
# How MLE evolves as we observe more data
n_flips = 10
observations = np.arange(n_flips + 1)  # 0 to n_flips observations

# Sequence 1: all heads
sequence1 = np.zeros(n_flips)
mle_seq1 = np.zeros(n_flips + 1)
mle_seq1[0] = 0.5  # Before any data, undefined (we'll use 0.5)

# Sequence 2: all tails
sequence2 = np.ones(n_flips)
mle_seq2 = np.zeros(n_flips + 1)
mle_seq2[0] = 0.5

for i in range(n_flips):
    sequence1[i] = 1  # Add a head
    sequence2[i] = 0  # Add a tail
    mle_seq1[i+1] = mle_estimate(sequence1[:i+1])
    mle_seq2[i+1] = mle_estimate(sequence2[:i+1])

# Plot
plt.figure(figsize=(8, 5))
plt.plot(observations, mle_seq1, 'b-o', linewidth=2, markersize=6, label='All heads')
plt.plot(observations, mle_seq2, 'r-o', linewidth=2, markersize=6, label='All tails')
plt.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
plt.grid(True, alpha=0.3)
plt.ylim(-0.05, 1.05)

# YOUR CODE HERE: Add xlabel, ylabel, title, legend


---

# 2. The Beta-Bernoulli Model

MLE has a problem: with few observations, it can give extreme estimates. If you flip a coin 3 times and get 3 heads, MLE says $\theta = 1$ — the coin always lands heads! This seems overconfident.

**Bayesian inference** addresses this by incorporating **prior beliefs**. We use:

- **Likelihood**: Bernoulli (each flip is independent with probability $\theta$)
- **Prior**: Beta distribution, $\text{Beta}(\alpha, \beta)$

The Beta distribution is a probability distribution over $\theta \in [0, 1]$. The parameters $\alpha$ and $\beta$ can be thought of as **pseudocounts**:
- $\alpha$ = number of "prior heads" we've imagined seeing + 1
- $\beta$ = number of "prior tails" we've imagined seeing + 1

Special cases:
- $\text{Beta}(1, 1)$ = Uniform (no prior preference)
- $\text{Beta}(2, 2)$ = Slight preference for $\theta = 0.5$
- $\text{Beta}(10, 10)$ = Strong belief that $\theta \approx 0.5$

In [ ]:
# Visualize different Beta priors
theta_values = np.linspace(0.001, 0.999, 200)

priors = [
    (1, 1, 'Uniform: Beta(1,1)'),
    (2, 2, 'Weak prior for 0.5: Beta(2,2)'),
    (5, 5, 'Moderate prior for 0.5: Beta(5,5)'),
    (10, 2, 'Prior favoring heads: Beta(10,2)'),
    (2, 10, 'Prior favoring tails: Beta(2,10)')
]

plt.figure(figsize=(10, 6))
for alpha, beta, label in priors:
    pdf = stats.beta(alpha, beta).pdf(theta_values)
    plt.plot(theta_values, pdf, linewidth=2, label=label)

plt.xlabel(r'$\theta$ (probability of heads)', fontsize=12)
plt.ylabel('Prior probability density', fontsize=12)
plt.title('Different Beta Prior Distributions', fontsize=14)
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)
plt.xlim(0, 1)
plt.show()

## The Posterior Distribution

After observing $k$ heads and $n-k$ tails, Bayes' rule gives us the **posterior distribution**:

$$P(\theta | \text{data}) \propto P(\text{data} | \theta) \cdot P(\theta)$$

For the Beta-Bernoulli model, the posterior is also a Beta distribution:

$$\text{Posterior} = \text{Beta}(\alpha + k, \beta + (n-k))$$

The prior pseudocounts simply get added to the observed counts!

In [ ]:
# Visualize how the posterior updates with data
# Prior: Beta(2, 2) - slight preference for fair coin
alpha_prior, beta_prior = 2, 2

# Observed data: 7 heads out of 10 flips
n_heads, n_tails = 7, 3

# Posterior parameters
alpha_post = alpha_prior + n_heads
beta_post = beta_prior + n_tails

theta_values = np.linspace(0.001, 0.999, 200)

plt.figure(figsize=(10, 6))

# Plot prior
prior_pdf = stats.beta(alpha_prior, beta_prior).pdf(theta_values)
plt.plot(theta_values, prior_pdf, 'b-', linewidth=2, 
         label=f'Prior: Beta({alpha_prior}, {beta_prior})')

# Plot likelihood (normalized for visualization)
likelihood = theta_values**n_heads * (1-theta_values)**n_tails
likelihood = likelihood / np.max(likelihood) * np.max(prior_pdf)  # Scale for visibility
plt.plot(theta_values, likelihood, 'g--', linewidth=2, 
         label=f'Likelihood (scaled): {n_heads}H, {n_tails}T')

# Plot posterior
posterior_pdf = stats.beta(alpha_post, beta_post).pdf(theta_values)
plt.plot(theta_values, posterior_pdf, 'r-', linewidth=2, 
         label=f'Posterior: Beta({alpha_post}, {beta_post})')

# Mark MLE
mle = n_heads / (n_heads + n_tails)
plt.axvline(x=mle, color='green', linestyle=':', alpha=0.7, label=f'MLE = {mle:.2f}')

plt.xlabel(r'$\theta$ (probability of heads)', fontsize=12)
plt.ylabel('Probability density', fontsize=12)
plt.title('Bayesian Updating: Prior × Likelihood ∝ Posterior', fontsize=14)
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.xlim(0, 1)
plt.show()

---

# 3. Maximum A Posteriori (MAP) Estimation

The **MAP estimate** is the value of $\theta$ that maximizes the posterior distribution:

$$\hat{\theta}_{MAP} = \frac{\alpha + k - 1}{\alpha + \beta + n - 2}$$

where $k$ = observed heads, $n$ = total flips, and $\alpha, \beta$ are the prior parameters. 

**Note**: When $\alpha = \beta = 1$ (uniform prior), MAP reduces to MLE.

<div class="alert alert-success">

## Exercise 2A

Implement a function that computes the MAP estimate of $\theta$.

</div>

In [ ]:
def map_estimate(sequence, alpha_parameter=1, beta_parameter=1):
    """Compute the Maximum A Posteriori estimate of theta.
    
    Parameters
    ----------
    sequence : numpy array of 1s and 0s
        The observed sequence of coin flips
    alpha_parameter : float
        Prior pseudocount for heads (prior_heads + 1)
    beta_parameter : float
        Prior pseudocount for tails (prior_tails + 1)
        
    Returns
    -------
    float
        The MAP estimate of theta
    """
    # YOUR CODE HERE


In [ ]:
# Test your implementation
sequence = np.array([1, 1, 0, 1, 0, 1, 1, 1, 1, 1])  # 8 heads, 2 tails

# With uniform prior (alpha=1, beta=1), MAP should equal MLE
assert map_estimate(sequence, 1, 1) == mle_estimate(sequence), "With uniform prior, MAP should equal MLE"

# Test with different priors
# MAP = (alpha + k - 1) / (alpha + beta + n - 2)
assert abs(map_estimate(sequence, 2, 2) - 9/12) < 0.01  # (2+8-1)/(2+2+10-2) = 9/12 = 0.75
assert abs(map_estimate(sequence, 5, 5) - 12/18) < 0.01  # (5+8-1)/(5+5+10-2) = 12/18
assert abs(map_estimate(np.array([]), 5, 5) - 0.5) < 0.01  # No data: (5-1)/(5+5-2) = 4/8 = 0.5

print("All tests passed!")

<div class="alert alert-success">

## Exercise 2B

The code below shows how MAP estimates evolve with data for different priors. 

**Your task**: Complete the figure by adding axis labels, title, and legend. Then answer: How does the strength of the prior affect how quickly the estimate converges to the MLE?

</div>

In [ ]:
# How different priors affect MAP estimates as data accumulates
n_flips = 20

# All heads sequence (extreme case)
sequence = np.ones(n_flips)

priors = [
    (1, 1, 'Uniform prior'),
    (2, 2, 'Weak prior (α=β=2)'),
    (10, 10, 'Strong prior (α=β=10)')
]

plt.figure(figsize=(10, 6))

for alpha, beta, label in priors:
    estimates = []
    for i in range(n_flips + 1):
        if i == 0:
            # Prior mode (before any data)
            est = (alpha - 1) / (alpha + beta - 2) if alpha > 1 and beta > 1 else 0.5
        else:
            est = map_estimate(sequence[:i], alpha, beta)
        estimates.append(est)
    plt.plot(range(n_flips + 1), estimates, 'o-', linewidth=2, markersize=5, label=label)

plt.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5, label='True θ = 1.0')
plt.xticks(range(0, n_flips + 1, 1))

# YOUR CODE HERE: Add xlabel, ylabel, title, legend


### YOUR ANSWER - Exercise 2B


---

# 4. Posterior Mean Estimation

Instead of finding the mode (peak) of the posterior, we can compute its **mean**:

$$\hat{\theta}_{PM} = \frac{\alpha + k}{\alpha + \beta + n}$$

Keep in mind that $\alpha$ = number of "prior heads" we've imagined seeing + 1 and $\beta$ = number of "prior tails" we've imagined seeing + 1

The posterior mean has nice properties:
- It minimizes expected squared error
- It's always well-defined (unlike MAP for uniform priors with no data)
- It's a weighted average between the prior mean and the MLE

<div class="alert alert-success">

## Exercise 3A

Implement a function that computes the posterior mean estimate of $\theta$.

</div>

In [ ]:
def posterior_mean(sequence, alpha_parameter=1, beta_parameter=1):
    """Compute the posterior mean estimate of theta.
    
    Parameters
    ----------
    sequence : numpy array of 1s and 0s
        The observed sequence of coin flips
    prior_heads : float
        Prior pseudocount for heads (alpha parameter)
    prior_tails : float
        Prior pseudocount for tails (beta parameter)
        
    Returns
    -------
    float
        The posterior mean estimate of theta
    """
    # YOUR CODE HERE


In [ ]:
# Test your implementation
sequence = np.array([1, 1, 0, 1, 0, 1, 1, 1, 1, 1])  # 8 heads, 2 tails

assert abs(posterior_mean(sequence, 1, 1) - 9/12) < 0.01  # (8+1)/(10+2)
assert abs(posterior_mean(sequence, 2, 2) - 10/14) < 0.01  # (8+2)/(10+4)
assert abs(posterior_mean(np.array([]), 5, 5) - 0.5) < 0.01  # Prior mean with no data
assert abs(posterior_mean(np.array([1, 1, 1, 1, 1, 0]), 1, 1) - 6/8) < 0.01

print("All tests passed!")

<div class="alert alert-success">

## Exercise 3B

Create a figure comparing MLE, MAP, and Posterior Mean estimates as data accumulates. Use a sequence that starts with 5 tails, then 15 heads. Use a moderate prior of Beta(5, 5).

**Your task**: Write the code to generate this comparison plot with proper labels.

</div>

In [ ]:
# YOUR CODE HERE

---

# 5. The Coin of Azeroth: Putting It All Together

Let's return to our original question. You flip the coin of Azeroth 10 times and observe:

**H H T H T H H H H H** (8 heads, 2 tails)

Based on prior experience with coins of Azeroth, you believe they tend to be slightly biased toward heads. You encode this as a prior of Beta(56, 46) — as if you've previously seen 100 flips with 55 heads.

In [ ]:
# The observed sequence
sequence = np.array([1, 1, 0, 1, 0, 1, 1, 1, 1, 1])

# Prior based on experience with Azeroth coins
alpha_parameter = 56
beta_parameter = 46

print("=" * 50)
print("COIN OF AZEROTH ANALYSIS")
print("=" * 50)
print(f"\nObserved: {int(np.sum(sequence))} heads, {int(len(sequence) - np.sum(sequence))} tails")
print(f"Prior: Beta({alpha_parameter}, {beta_parameter}) — prior mean = {alpha_parameter/(alpha_parameter+beta_parameter):.3f}")
print("\n" + "-" * 50)
print(f"{'Method':<20} {'Estimate':>15}")
print("-" * 50)
print(f"{'MLE':<20} {mle_estimate(sequence):>15.4f}")
print(f"{'MAP':<20} {map_estimate(sequence, alpha_parameter, beta_parameter):>15.4f}")
print(f"{'Posterior Mean':<20} {posterior_mean(sequence, alpha_parameter, beta_parameter):>15.4f}")
print("-" * 50)
print(f"\nFather's requirement: θ between 0.55 and 0.75")

<div class="alert alert-success">

## Exercise 4

Based on the results above:

1. Should you take this coin for your father? Why or why not?

2. The MLE and Bayesian estimates give very different answers. Why is this? Which do you trust more in this situation?

3. If you could flip the coin 100 more times, would you expect the three estimates to become more similar or more different? Explain.

</div>

### YOUR ANSWERS - Exercise 4

---

# 6. Visualizing Uncertainty: The Full Posterior (Optional)
## Please note: this section is optional, but there is a Part 2 of the problem set below that is not optional!

Point estimates (MLE, MAP, posterior mean) give us a single number. But the full posterior distribution tells us much more — it shows our **uncertainty** about $\theta$.

In [ ]:
# Visualize the full posterior for the coin of Azeroth
sequence = np.array([1, 1, 0, 1, 0, 1, 1, 1, 1, 1])
n_heads = int(np.sum(sequence))
n_tails = len(sequence) - n_heads

# Prior and posterior parameters
alpha_prior, beta_prior = 56, 46
alpha_post = alpha_prior + n_heads
beta_post = beta_prior + n_tails

theta = np.linspace(0, 1, 500)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left plot: Prior vs Posterior
ax = axes[0]
ax.plot(theta, stats.beta(alpha_prior, beta_prior).pdf(theta), 'b-', linewidth=2, label='Prior')
ax.plot(theta, stats.beta(alpha_post, beta_post).pdf(theta), 'r-', linewidth=2, label='Posterior')
ax.axvline(x=posterior_mean(sequence, alpha_prior, beta_prior), color='r', linestyle='--', 
           alpha=0.7, label=f'Post. Mean = {posterior_mean(sequence, alpha_prior, beta_prior):.3f}')
ax.axvspan(0.55, 0.75, alpha=0.2, color='green', label="Father's range [0.55, 0.75]")
ax.set_xlabel(r'$\theta$', fontsize=12)
ax.set_ylabel('Probability density', fontsize=12)
ax.set_title('Prior and Posterior Distributions', fontsize=13)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)
ax.set_xlim(0.3, 0.9)

# Right plot: Posterior with credible interval
ax = axes[1]
posterior = stats.beta(alpha_post, beta_post)
ax.plot(theta, posterior.pdf(theta), 'r-', linewidth=2)

# 95% credible interval
ci_low, ci_high = posterior.ppf(0.025), posterior.ppf(0.975)
theta_ci = theta[(theta >= ci_low) & (theta <= ci_high)]
ax.fill_between(theta_ci, posterior.pdf(theta_ci), alpha=0.3, color='red',
                label=f'95% CI: [{ci_low:.3f}, {ci_high:.3f}]')

ax.axvspan(0.55, 0.75, alpha=0.2, color='green', label="Father's range")
ax.set_xlabel(r'$\theta$', fontsize=12)
ax.set_ylabel('Probability density', fontsize=12)
ax.set_title('Posterior with 95% Credible Interval', fontsize=13)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)
ax.set_xlim(0.3, 0.9)

plt.tight_layout()
plt.show()

# Calculate probability that theta is in father's range
prob_in_range = posterior.cdf(0.75) - posterior.cdf(0.55)
print(f"\nProbability that θ is in [0.55, 0.75]: {prob_in_range:.3f} ({prob_in_range*100:.1f}%)")

<div class="alert alert-success">

## Exercise 5

Explore how the posterior changes with different priors and data:

1. Create a figure showing posteriors for the same data (8H, 2T) but with three different priors: weak (α=β=2), moderate (α=β=10), and strong (α=β=50, centered at 0.5). How does prior strength affect uncertainty?

2. Create a figure showing how the posterior evolves as you observe more data. Start with the Beta(56,46) prior, and show the posterior after 10, 50, and 200 flips (assume 70% heads throughout).

</div>

In [ ]:
# YOUR CODE HERE - Exercise 5.1: Different prior strengths

In [ ]:
# YOUR CODE HERE - Exercise 5.2: Posterior evolution with more data

### YOUR NOTES - Exercise 5

---

# Part 2: Bayesian Occam's Razor

In Part 1, we estimated a continuous parameter ($\theta$) from data. Now we'll explore a different problem: **hypothesis selection**. Given some observations, which hypothesis best explains the data?

Bayesian inference automatically implements **Occam's Razor** — the principle that simpler explanations should be preferred, all else being equal. But "simpler" has a precise meaning: hypotheses that make **sharper predictions** (assign probability to fewer outcomes) are rewarded when those predictions are correct.

## 7.1 The Image Classification Problem

Imagine you're trying to learn a concept from examples, like the example in class. A teacher shows you images, and you must figure out what concept they're teaching.

We have **12 images** with nested features:
- Images 1-12: All are outdoor scenes
- Images 1-9: Include a mammal
- Images 1-6: Include a large mammal  
- Images 1-3: Include a horse
- Images 1-2: Include a **running** horse

Each feature corresponds to a hypothesis about the concept the teacher might be teaching.

In [ ]:
# Visualize the nested structure of hypotheses
import matplotlib.patches as mpatches

fig, ax = plt.subplots(figsize=(12, 6))

# Define the hypotheses and which images they cover
hypotheses = {
    'Outdoors': list(range(1, 13)),      # Images 1-12
    'Mammal': list(range(1, 10)),         # Images 1-9
    'Large mammal': list(range(1, 7)),    # Images 1-6
    'Horse': list(range(1, 4)),           # Images 1-3
    'Running horse': list(range(1, 3))    # Images 1-2
}

colors = ['#E8E8E8', '#FFD700', '#FFA500', '#FF6347', '#DC143C']
y_positions = [0, 1, 2, 3, 4]

# Draw rectangles for each hypothesis
for (name, images), color, y in zip(hypotheses.items(), colors, y_positions):
    width = len(images)
    rect = mpatches.FancyBboxPatch((0.5, y - 0.35), width, 0.7, 
                                    boxstyle="round,pad=0.02", 
                                    facecolor=color, edgecolor='black', linewidth=2)
    ax.add_patch(rect)
    ax.text(width/2 + 0.5, y, f'{name}\n({len(images)} images)', 
            ha='center', va='center', fontsize=11, fontweight='bold')

# Draw image markers
for i in range(1, 13):
    ax.plot(i, -1, 's', markersize=20, color='lightblue', markeredgecolor='black')
    ax.text(i, -1, str(i), ha='center', va='center', fontsize=10, fontweight='bold')

ax.text(6.5, -1.8, 'Images', ha='center', fontsize=12)

ax.set_xlim(0, 13)
ax.set_ylim(-2.5, 5)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('Nested Hypothesis Structure', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 7.2 The Key Insight: Specificity vs. Risk

Each hypothesis defines a **likelihood function** — the probability of observing each image given that hypothesis:

$$P(\text{image } i \mid H) = \begin{cases} \frac{1}{|H|} & \text{if image } i \in H \\ 0 & \text{otherwise} \end{cases}$$

where $|H|$ is the number of images consistent with hypothesis $H$.

**The tradeoff:**
- **Specific hypotheses** (like "running horse") assign high probability to each consistent image (1/2 each)
- **General hypotheses** (like "outdoors") spread probability thinly (1/12 each)

But specific hypotheses are **riskier** — they're wrong for more images!

In [ ]:
# Define the hypotheses
hypotheses = {
    'Outdoors': set(range(1, 13)),       # Images 1-12
    'Mammal': set(range(1, 10)),          # Images 1-9
    'Large mammal': set(range(1, 7)),     # Images 1-6
    'Horse': set(range(1, 4)),            # Images 1-3
    'Running horse': set(range(1, 3))     # Images 1-2
}

def likelihood(image, hypothesis_name):
    """P(image | hypothesis) - probability of seeing this image given the hypothesis."""
    h_images = hypotheses[hypothesis_name]
    if image in h_images:
        return 1.0 / len(h_images)  # Uniform over consistent images
    else:
        return 0.0  # Impossible under this hypothesis

# Show likelihoods for a few example images
print("Likelihood P(image | hypothesis) for different images:\n")
print(f"{'Hypothesis':<15} {'Image 1':>10} {'Image 2':>10} {'Image 3':>10} {'Image 7':>10}")
print("-" * 55)
for h in hypotheses.keys():
    l1 = likelihood(1, h)
    l2 = likelihood(2, h)
    l3 = likelihood(3, h)
    l7 = likelihood(7, h)
    print(f"{h:<15} {l1:>10.3f} {l2:>10.3f} {l3:>10.3f} {l7:>10.3f}")

## 7.3 Bayesian Hypothesis Testing

Given observed images, we update our beliefs about which hypothesis is correct using Bayes' rule:

$$P(H \mid \text{data}) \propto P(\text{data} \mid H) \cdot P(H)$$

We'll start with a **uniform prior** (all hypotheses equally likely) and see how observations shift our beliefs.

<div class="alert alert-success">

## Exercise 6

Implement the `compute_posterior` function below to calculate the posterior probability over hypotheses given observed images.

**Steps:**
1. Compute the likelihood of all observations for each hypothesis (multiply the individual likelihoods)
2. Compute the unnormalized posterior = likelihood × prior
3. Normalize so the posteriors sum to 1

**Hint:** Use the `likelihood(image, hypothesis_name)` function defined above.

</div>

In [ ]:
def compute_posterior(observed_images, prior=None):
    """Compute posterior probability over hypotheses given observed images.
    
    Parameters
    ----------
    observed_images : list of int
        The images that were observed
    prior : dict, optional
        Prior probability for each hypothesis. Default is uniform.
    
    Returns
    -------
    posterior : dict
        Posterior probability for each hypothesis
    likelihoods : dict
        Likelihood of the data for each hypothesis
    """
    if prior is None:
        prior = {h: 1/len(hypotheses) for h in hypotheses}
    
    # YOUR CODE HERE


def plot_posterior(observed_images, ax=None):
    """Plot the posterior distribution over hypotheses."""
    posterior, likelihoods = compute_posterior(observed_images)
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 5))
    
    h_names = list(hypotheses.keys())
    probs = [posterior[h] for h in h_names]
    colors = ['#E8E8E8', '#FFD700', '#FFA500', '#FF6347', '#DC143C']
    
    bars = ax.bar(h_names, probs, color=colors, edgecolor='black', linewidth=2)
    ax.set_ylabel('Posterior Probability', fontsize=12)
    ax.set_ylim(0, 1)
    ax.set_title(f'Observed: Image(s) {observed_images}', fontsize=13)
    
    # Add probability labels on bars
    for bar, prob in zip(bars, probs):
        if prob > 0.01:
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
                   f'{prob:.2f}', ha='center', fontsize=10)
    
    return posterior

In [ ]:
# Test your implementation
posterior, likelihoods = compute_posterior([1])

# Check that posterior sums to 1
assert abs(sum(posterior.values()) - 1.0) < 0.001, "Posterior should sum to 1"

# Check specific values for observing image 1
# Running horse has likelihood 1/2, should have highest posterior
assert posterior['Running horse'] > posterior['Horse'], "Running horse should beat Horse for image 1"
assert posterior['Horse'] > posterior['Large mammal'], "Horse should beat Large mammal for image 1"

# Check that likelihoods are computed correctly
assert abs(likelihoods['Running horse'] - 0.5) < 0.001, "P(image 1 | Running horse) = 1/2"
assert abs(likelihoods['Horse'] - 1/3) < 0.001, "P(image 1 | Horse) = 1/3"
assert abs(likelihoods['Outdoors'] - 1/12) < 0.001, "P(image 1 | Outdoors) = 1/12"

# Test with image 3 (which rules out Running horse)
posterior3, likelihoods3 = compute_posterior([3])
assert likelihoods3['Running horse'] == 0, "Image 3 is not consistent with Running horse"
assert posterior3['Running horse'] == 0, "Running horse posterior should be 0 for image 3"
assert posterior3['Horse'] > posterior3['Large mammal'], "Horse should have highest posterior for image 3"

print("All tests passed!")

## 7.4 Example: Single Observation

Let's see what happens when we observe a single image. Watch how the **most specific consistent hypothesis** gets the highest posterior!

In [ ]:
# Compare posteriors for different single observations
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

observations = [[1], [2], [3], [7]]
titles = ['Image 1 (running horse)', 'Image 2 (running horse)', 
          'Image 3 (horse, not running)', 'Image 7 (mammal, not large)']

for ax, obs, title in zip(axes.flat, observations, titles):
    plot_posterior(obs, ax)
    ax.set_title(title, fontsize=11)
    ax.tick_params(axis='x', rotation=30)

plt.tight_layout()
plt.show()

print("\nNotice: The most SPECIFIC hypothesis consistent with the data wins!")
print("This is Bayesian Occam's Razor in action.")

## 7.5 Multiple Observations: Evidence Accumulates

With multiple observations, the posterior reflects which hypothesis best explains **all** the data. Hypotheses that are inconsistent with even one observation get eliminated.

In [ ]:
# Show how posterior evolves with multiple observations
fig, axes = plt.subplots(2, 3, figsize=(14, 8))

# Scenario 1: Observing images 1, then 2 (both running horses)
observation_sequences = [
    ([1], 'After seeing image 1'),
    ([1, 2], 'After seeing images 1, 2'),
    ([1, 2, 1], 'After seeing images 1, 2, 1'),
]

# Scenario 2: Observing images 1, then 3 (running horse rules out)
observation_sequences2 = [
    ([1], 'After seeing image 1'),
    ([1, 3], 'After seeing images 1, 3'),
    ([1, 3, 2], 'After seeing images 1, 3, 2'),
]

for i, (obs, title) in enumerate(observation_sequences):
    plot_posterior(obs, axes[0, i])
    axes[0, i].set_title(title, fontsize=10)
    axes[0, i].tick_params(axis='x', rotation=30)

for i, (obs, title) in enumerate(observation_sequences2):
    plot_posterior(obs, axes[1, i])
    axes[1, i].set_title(title, fontsize=10)
    axes[1, i].tick_params(axis='x', rotation=30)

axes[0, 0].set_ylabel('Scenario 1\n(all running horses)\n\nPosterior Prob.', fontsize=10)
axes[1, 0].set_ylabel('Scenario 2\n(image 3 rules out\nrunning horse)\n\nPosterior Prob.', fontsize=10)

plt.tight_layout()
plt.show()

## 7.6 Example: The Three Horse Images

Suppose the teacher shows you images 1, 2, and 3 — exactly the three horse images. What should you conclude?

Let's compute the posterior step by step:

In [ ]:
# The teacher shows images 1, 2, and 3 (all three horse images)
observed = [1, 2, 3]

# Compute posterior
posterior, likelihoods = compute_posterior(observed)

# Show the likelihood calculation
print("Step 1: Compute likelihood P(data | H) for each hypothesis")
print("=" * 60)
print(f"Data observed: images {observed}\n")

for h, h_images in hypotheses.items():
    n = len(h_images)
    # Check if all observed images are in this hypothesis
    all_consistent = all(img in h_images for img in observed)
    
    if all_consistent:
        lik = (1/n) ** len(observed)
        print(f"{h:<15}: Each image has P = 1/{n}")
        print(f"               Likelihood = (1/{n})^3 = {lik:.6f}")
    else:
        print(f"{h:<15}: Image 3 is NOT in this set → Likelihood = 0")
    print()

print("\nStep 2: Apply Bayes' rule (uniform prior)")
print("=" * 60)
print("P(H | data) ∝ P(data | H) × P(H)")
print("With uniform prior, posterior ∝ likelihood\n")

# Plot
fig, ax = plt.subplots(figsize=(10, 5))
plot_posterior(observed, ax)
ax.set_title('Posterior after observing images 1, 2, 3 (the three horses)', fontsize=12)
plt.tight_layout()
plt.show()

print(f"\nPosterior probabilities:")
print("-" * 40)
for h in hypotheses:
    print(f"{h:<15}: {posterior[h]:.4f}")
    
print(f"\n→ The 'Horse' hypothesis wins with {posterior['Horse']:.1%} probability!")
print("  It's the most SPECIFIC hypothesis consistent with ALL the data.")

<div class="alert alert-success">

## 7.7 Exercise 7

Explore the Bayesian Occam's razor effect:

1. What happens if you observe images [4, 5, 6]? Which hypothesis wins? Why?

2. What's the minimum number of observations needed to be 90% confident in the "Horse" hypothesis?

3. Suppose the teacher is actually teaching "Large mammal". Simulate observing random images from this concept (images 1-6) and plot how the posterior evolves. How many observations does it typically take to correctly identify the hypothesis?

</div>

In [ ]:
# YOUR CODE HERE - Exercise 7

### YOUR ANSWERS - Exercise 7

## 7.8 Why Does Occam's Razor Work?

The Bayesian explanation is elegant:

1. **Specific hypotheses make sharper predictions** — they assign higher probability to each consistent outcome
2. **When the prediction is correct**, this higher likelihood translates to higher posterior probability
3. **But specific hypotheses are also riskier** — they rule out more possibilities

The math automatically balances these tradeoffs. You don't need to manually penalize complex hypotheses — the likelihood function does it for you!

$$\underbrace{P(H|\text{data})}_{\text{posterior}} \propto \underbrace{P(\text{data}|H)}_{\text{rewards specificity}} \times \underbrace{P(H)}_{\text{prior}}$$

This is sometimes called the **size principle**: among hypotheses consistent with the data, prefer the smallest (most specific) one.